#### imports to enable analysis

In [ ]:
import json
from pyspark.sql import Row
from pyspark.sql import functions as F 
from pyspark.sql.functions import from_json, col, lit, countDistinct, avg, col
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, LongType
#import org.apache.spark.sql.functions.countDistinct
import sys 
from pyspark.sql.window import Window

#### Read assessments data from kafka into a pyspark dataframe

In [ ]:
raw_assessments = spark.read.format("kafka").option("kafka.bootstrap.servers", "kafka:29092").option("subscribe","assessments").option("startingOffsets", "earliest").option("endingOffsets", "latest").load() 

In [ ]:
type(raw_assessments)

#### Cache the dataframe to cut back on warnings

In [264]:
raw_assessments.cache()

DataFrame[key: binary, value: binary, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]

#### Check count to make sure data was written properly

In [265]:
raw_assessments.count()

3280

#### Cast to strings

In [ ]:
assessments = raw_assessments.select(raw_assessments.value.cast('string'))

#### Extract json fields

In [266]:
extracted_assessments = assessments.rdd.map(lambda x: Row(**json.loads(x.value))).toDF()


In [267]:
type(extracted_assessments)

pyspark.sql.dataframe.DataFrame

#### Take a look at the new dataframe to get a sense of its structure

In [268]:
extracted_assessments.show(5)

+--------------------+-------------+--------------------+------------------+--------------------+------------------+------------+--------------------+--------------------+--------------------+
|        base_exam_id|certification|           exam_name|   keen_created_at|             keen_id|    keen_timestamp|max_attempts|           sequences|          started_at|        user_exam_id|
+--------------------+-------------+--------------------+------------------+--------------------+------------------+------------+--------------------+--------------------+--------------------+
|37f0a30a-7464-11e...|        false|Normal Forms and ...| 1516717442.735266|5a6745820eb8ab000...| 1516717442.735266|         1.0|Map(questions -> ...|2018-01-23T14:23:...|6d4089e4-bde5-4a2...|
|37f0a30a-7464-11e...|        false|Normal Forms and ...| 1516717377.639827|5a674541ab6b0a000...| 1516717377.639827|         1.0|Map(questions -> ...|2018-01-23T14:21:...|2fec1534-b41f-441...|
|4beeac16-bb83-4d5...|        false

#### Look at the structure by looking at the schema:

In [269]:
extracted_assessments.printSchema()

root
 |-- base_exam_id: string (nullable = true)
 |-- certification: string (nullable = true)
 |-- exam_name: string (nullable = true)
 |-- keen_created_at: string (nullable = true)
 |-- keen_id: string (nullable = true)
 |-- keen_timestamp: string (nullable = true)
 |-- max_attempts: string (nullable = true)
 |-- sequences: map (nullable = true)
 |    |-- key: string
 |    |-- value: array (valueContainsNull = true)
 |    |    |-- element: map (containsNull = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: boolean (valueContainsNull = true)
 |-- started_at: string (nullable = true)
 |-- user_exam_id: string (nullable = true)



#### Only sequences is really nested

#### Create a temp table to begin to unnest

In [ ]:
extracted_assessments.registerTempTable('assessments')

#### Look at unnested columns first

In [ ]:
spark.sql("select base_exam_id, certification, exam_name, keen_created_at from assessments limit 5").show()

In [ ]:
spark.sql("select keen_id, keen_timestamp, max_attempts, started_at, user_exam_id from assessments limit 5").show()

In [ ]:
spark.sql("select sequences from assessments limit 5").show()

There's no columns, at face value, I'd throw out. I could see how they could all have analytical value.

In [ ]:
sequences_df = spark.sql("select sequences from assessments")

In [ ]:
sequences_df.select('sequences').take(1)

In [ ]:
sequences_df.select('sequences').take(1)[0]

In [ ]:
sequences_df.select('sequences').take(1)[0][0]

In [ ]:
sequences_df.select('sequences').take(1)[0][0]['questions']

In [ ]:
sequences_df.select('sequences').take(1)[0][0]['questions'][0]

In [ ]:
sequences_df.select('sequences').take(1)[0][0]['questions'][0]['user_incomplete']

In [ ]:
assessments_df = spark.read.json(assessments.rdd.map(lambda x: x.value))

In [ ]:
spark.sql("select base_exam_id, certification, exam_name, keen_created_at from assessments limit 5").show()

In [ ]:
count_df.select("counts.correct", "counts.incorrect","counts.incomplete","counts.unanswered","counts.submitted", "counts.total" ).show()

In [ ]:
count_df.select("counts.correct", "counts.incorrect","counts.incomplete","counts.unanswered","counts.submitted", "counts.total" ).show()

In [ ]:
assessments_df.select("base_exam_id", "sequences.counts.correct").show()

In [ ]:
assessments_df.printSchema()

In [ ]:
assessments_df.select('sequences.questions.options').take(1)[0][0][0][0]

In [ ]:
assessments_df.select('sequences').take(1)[0][0]['questions'][0]['user_incomplete']

In [ ]:
assessments_df.select('sequences.questions').take(1)[0][0]

In [ ]:
#assessments_df.select('sequences.questions').show()

In [ ]:
# are these the IDs of the questions taken?
assessments_df.select('sequences.questions.id').take(1)[0][0]

In [ ]:
# This seems to tie out the # of IDs to the total number of questions (below)
assessments_df.select(F.size('sequences.questions.id')).show()

In [ ]:
# are these the IDs of the questions taken?
assessments_df.select('sequences.questions.id').count()

In [ ]:
assessments_df.select('sequences.counts.total', 'sequences.counts.correct', 'sequences.counts.incorrect').show()

In [ ]:
assessments_df.select('sequences.counts').show()

In [ ]:
# Just print portion of schema associated with counts

In [ ]:
assessments_df.select('sequences.counts').printSchema()

In [ ]:
count_df = assessments_df.select('sequences.counts')


### JOIN THESE TO MAIN TABLE


In [ ]:
count_df.select("counts.correct", "counts.incorrect","counts.incomplete","counts.unanswered","counts.submitted", "counts.total" ).show()

### 1. How many assessments are in the dataset?

In [ ]:
assessments_df.count()

If each row is an assessment, then there were 3,280 assessments in the dataset. 

### 2. What's the name of your Kafka topic? How did you come up with that name?

My topic name was 'assessments'. That seemed like a descriptive name. 

In [ ]:
If each row is an assessment, then there were 3,280 assessments in the dataset. 

### 3. How many people took Learning Git?

In [ ]:
assessments_df.filter(assessments_df["exam_name"] == "Learning Git").count()

394 people took 'Learning Git'

### 4a. What is the least common course taken?

In [ ]:
assessments_df.select('exam_name').groupBy("exam_name").count().show()

In [ ]:
assessments_df.select('exam_name').groupBy("exam_name").count().agg({'count':'max'}).show()

In [ ]:
assessments_df.select('exam_name').groupBy("exam_name").count().agg({'count':'min'}).show()

In [ ]:
assessments_df.select('exam_name').groupBy("exam_name").count().sort("count", ascending = False).collect()

In [ ]:
course_counts = assessments_df.select('exam_name').groupBy("exam_name").count()

In [ ]:
max_course = assessments_df.select('exam_name').groupBy("exam_name").count().agg({'count':'max'})
min_course = assessments_df.select('exam_name').groupBy("exam_name").count().agg({'count':'min'})

In [ ]:
max_course.collect()[0][0]

In [ ]:
min_course.collect()[0][0]

In [ ]:
course_counts.filter(course_counts["count"] == max_course.collect()[0][0] ).show()

In [ ]:
course_counts.filter(course_counts["count"] == min_course.collect()[0][0] ).show(course_counts.count(), False)

In [ ]:
course_counts.filter(course_counts["count"] == min_course.collect()[0][0] ).select('exam_name').show(course_counts.count(), False)

### 5. How would I determine the score (= percent correct) for each assessment?

In [ ]:
assessments_df.select( (  (col("sequences.counts.correct")/col("sequences.counts.total")).alias("pct_correct"))).show(10)

### 6. How many unique exams are offered?

In [ ]:
assessments_df.select(countDistinct("exam_name")).show()



### 7. What's the average score on all assessments?

### 8. What percent of assessments are certified?

In [ ]:
#mean / avg combined with when:

#from pyspark.sql.functions import avg, col, when

#assessments_df.groupBy("certification").agg(F.avg(F.when(F.col("certification") == "true", 1).otherwise(0))).show()


null_assess = assessments_df.filter(assessments_df["certification"].isNull()).select('certification').count()
false_assess = assessments_df.filter(assessments_df["certification"] == 'false').select('certification').count()
total_assess = assessments_df.count()

print("Number of assessments with certification == 'false' =", false_assess)
print("Number of assessments with certification == 'NULL' =", null_assess)
print("Total assessment =", total_assess)


It turns out that the 'certification' field is either 'false' or NULL. So 0% of assessments in the database are for certified assessments. In the ordinary course of work, I'd dig deeper to determine if this is a worthwhile field to keep. 

In [ ]:
assessments_df.select("exam_name").show()

In [ ]:
spark.sql("select count(*)  from assessments_df")